In [53]:
%matplotlib inline
import bids
import mne
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd

import sys
sys.path.append('/Users/rdgao/Documents/code/research/spectralCV/')
sys.path.append('/Users/rdgao/Documents/code/research/neurodsp/')

from scv_funcs import lfpca

In [236]:
subj, sess, run = '01', '01', 1
layout = bids.layout.BIDSLayout('/Users/rdgao/Documents/data/Hermes/')
f_raw = layout.get(return_type='object', suffix='ieeg', subject=subj, session=sess, run=run, extensions='.vhdr')[0]
f_events = layout.get(return_type='object', suffix='events', subject=subj, session=sess, run=run)[0]
f_channels = layout.get(return_type='object', suffix='channels', subject=subj, session=sess, run=run)[0]
f_elec = layout.get(return_type='object', suffix='electrodes', subject=subj, session=sess)[0]

events = pd.read_csv(f_events.path, delimiter='\t')
chan_info = pd.read_csv(f_channels.path, delimiter='\t')
elec_loc = pd.read_csv(f_elec.path, delimiter='\t').apply(pd.to_numeric, errors='coerce')
#elec_loc = elec_loc.apply(pd.to_numeric, errors='coerce')


In [237]:
layout.get_metadata(f_raw.path)

{'ECGChannelCount': 0,
 'ECOGChannelCount': 118,
 'EEGChannelCount': 0,
 'EMGChannelCount': 0,
 'EOGChannelCount': 0,
 'ElectrodeManufacturer': 'AdTech',
 'EpochLength': 0,
 'HardwareFilters': ['highpass', '0.5', 'lowpass', '300'],
 'InstitutionAddress': '300 Pasteur Dr, Stanford, CA 94305',
 'InstitutionName': 'Stanford Hospital and Clinics',
 'Instructions': 'look at the dot in the center of the screen and press the button when it changes color',
 'Manufacturer': 'Tucker Davis Technologies',
 'MiscChannelCount': 0,
 'PowerLineFrequency': 60,
 'RecordingDuration': 233.639,
 'RecordingType': 'continuous',
 'SEEGChannelCount': 0,
 'SamplingFrequency': 3051.76,
 'SoftwareFilters': 'n/a',
 'TaskDescription': 'visual gratings and noise patterns',
 'TaskName': 'visual',
 'TriggerChannelCount': 0,
 'iEEGPlacementScheme': 'right occipital temporal surface',
 'iEEGReferenceScheme': 'intracranial channel not included with data'}

In [ ]:
raw = mne.io.read_raw_brainvision(f_raw.path, preload=True)
raw.info
data, t = raw[:,:]
data = data[:-1,:]

Extracting parameters from /Users/rdgao/Documents/data/Hermes/sub-01/ses-01/ieeg/sub-01_ses-01_task-visual_run-01_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 713009  =      0.000 ...   233.620 secs...


# Run SCV

In [ ]:
fs = 1./np.diff(t).mean()
params = {'nperseg':int(fs), 'noverlap':0, 'spg_outlierpct':1, 'max_freq':250}
lf = lfpca.LFPCA(params)
lf.populate_ts_data(data,fs)
lf.compute_all_spectral()
lf.compute_KS_expfit()

In [ ]:
good_chans = np.where(chan_info['status']=='good')[0]
plt.figure(figsize=(12,4))
lf.plot_spectral(plot_mean=False, plot_chan=good_chans, exc_freqs=[[56,65]])

In [ ]:
f_range = (30,50)
scv_mean = lf.scv[good_chans,f_range[0]:f_range[1]].mean(axis=1)
plt.figure(figsize=(6,6))
plt.scatter(elec_loc['x'][good_chans], elec_loc['y'][good_chans], s=100, c=scv_mean, alpha=0.8)
plt.colorbar(label='Mean SCV (%i-%iHz)'%f_range)